In [1]:
import json
from pathlib import Path
import numpy as np
from copy import deepcopy
import pandas as pd

from deeppavlov.core.commands.train import read_data_by_config, train_evaluate_model_from_config
from deeppavlov.core.commands.infer import interact_model, build_model_from_config
from deeppavlov.core.commands.utils import expand_path
from deeppavlov.core.common.params import from_params
from deeppavlov.core.common.errors import ConfigError

In [2]:
# read unlabelled data for label propagation
def read_unlabelled_data(UNLABELLED_DATA_PATH):
    with open(UNLABELLED_DATA_PATH, "r") as f:
        unlabelled_data = f.read().splitlines()
    unlabelled_data = [x for x in unlabelled_data if x != '']
    return unlabelled_data

In [3]:
def make_pl_config(CONFIG_PATH):
    config_path_pl = Path(CONFIG_PATH).parent / Path(Path(CONFIG_PATH).stem + "_pl.json")

    with open(CONFIG_PATH, "r") as f:
        config = json.load(f)
    
    config_pl = deepcopy(config)
    config_pl["dataset_reader"]["train"] = Path(config_pl["dataset_reader"].get("train", "train.csv")).stem + "_pl.csv"
    
    with open(config_path_pl, "w") as f:
        json.dump(config_pl, f, indent=2)
    
    return config, config_pl

In [4]:
def save_extended_data(config, samples, labels, new_config = None):
    train_data = read_data_by_config(deepcopy(config))
    
    for i in range(len(samples)):
        train_data["train"].append((samples[i], labels[i]))
    df = pd.DataFrame(train_data["train"], 
                      columns=[config["dataset_reader"]["x"], 
                               config["dataset_reader"]["y"]])
    df[config["dataset_reader"]["y"]] = df[config["dataset_reader"]["y"]].apply(
        lambda x: config["dataset_reader"].get("class_sep", ",").join(x))
    
    if new_config is not None:
        config = new_config
    file = expand_path(Path(config["dataset_reader"]["data_path"]) / 
                       Path(config["dataset_reader"]["train"]))

    if config["dataset_reader"].get("format", "csv") == "csv":
        keys = ('sep', 'header', 'names')
        df.to_csv(file, 
                  index=False,
                  sep=config["dataset_reader"].get("sep", ",")
                 )
    elif config["dataset_reader"].get("format", "csv") == "json":
        keys = ('orient', 'lines')
        df.to_json(file, 
                  index=False,
                  orient=config["dataset_reader"].get("orient", None),
                  lines=config["dataset_reader"].get("lines", False)
                  )
    else:
        raise ConfigError("Can not work with current data format")

In [8]:
# manually given parameters for pseudo-labeling

# path to config file
CONFIG_PATH = "../deeppavlov/configs/classifiers/yahoo_answers_L31_elmo_pretrained_n9.json"
# path to file with unlabelled data
UNLABELLED_DATA_PATH = "../download/YahooAnswers/yahoo_answers_data/question_L6.txt"
# number of samples that are going to be labelled during one iteration of label propagation
ONE_ITERATION_PORTION = 2000
# number of iterations
N_ITERATIONS = 10
CLASSES_VOCAB_ID_IN_PIPE = 0
CONFIDENT_PROBA = 0.9

In [9]:
# read unlabelled dataset
unlabelled_data = read_unlabelled_data(UNLABELLED_DATA_PATH)
# read config, compose new one, save it
config, config_pl = make_pl_config(CONFIG_PATH)
# save initial dataset as extended
save_extended_data(config, [], [], new_config=config_pl)

In [10]:
config

{'dataset_reader': {'name': 'basic_classification_reader',
  'x': 'Title',
  'y': 'Label',
  'data_path': './YahooAnswers/yahoo_answers_data/',
  'train': 'train.csv',
  'valid': 'valid.csv'},
 'dataset_iterator': {'name': 'basic_classification_iterator', 'seed': 42},
 'chainer': {'in': ['x'],
  'in_y': ['y'],
  'pipe': [{'id': 'classes_vocab',
    'name': 'simple_vocab',
    'fit_on': ['y'],
    'save_path': 'YahooAnswers/models/model_v39/yahoo_answers_classes.dict',
    'load_path': 'YahooAnswers/models/model_v39/yahoo_answers_classes.dict',
    'in': 'y',
    'out': 'y_ids'},
   {'in': ['x'],
    'out': ['x_prep'],
    'name': 'dirty_comments_preprocessor',
    'remove_punctuation': False},
   {'in': 'x_prep',
    'out': 'x_tok',
    'id': 'my_tokenizer',
    'name': 'nltk_moses_tokenizer'},
   {'in': ['x_tok'],
    'out': ['x_emb'],
    'id': 'my_embedder',
    'name': 'elmo',
    'elmo_output_names': ['elmo'],
    'mini_batch_size': 8,
    'spec': '/home/kuznetsov/models_env/yahoo

In [ ]:
available_unlabelled_ids = np.arange(len(unlabelled_data))

np.random.seed(42)

for i in range(N_ITERATIONS):
    samples = []
    labels = []
    
    ids_to_label = available_unlabelled_ids[
        np.random.randint(low=0, 
                          high=len(available_unlabelled_ids), 
                          size=ONE_ITERATION_PORTION)]
    available_unlabelled_ids = np.delete(available_unlabelled_ids, ids_to_label)
    train_evaluate_model_from_config(deepcopy(config_pl))
    model = build_model_from_config(deepcopy(config_pl))
    classes = np.array(list(from_params(
        deepcopy(config_pl["chainer"]["pipe"][CLASSES_VOCAB_ID_IN_PIPE])).keys()))

    for j, sample_id in enumerate(ids_to_label):
        prediction = model([unlabelled_data[sample_id]])[0]
        if len(np.where(np.array(prediction) > CONFIDENT_PROBA)[0]):
            samples.append(unlabelled_data[sample_id])
            labels.append(classes[np.where(np.array(prediction) > CONFIDENT_PROBA)])
    
    print("Iteration {}: add {} samples to train dataset".format(i, len(samples)))
    save_extended_data(config_pl, samples, labels)

2018-11-16 10:57:38.847 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 89: [saving vocabulary to /cephfs/home/dilyara.baymurzina/GitHub/reserve/DeepPavlov/download/YahooAnswers/models/model_v39/yahoo_answers_classes.dict]
2018-11-16 10:57:41.645 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/aggregation/elmo_output_ELMo_W:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with aggregation/elmo_output_ELMo_W
2018-11-16 10:57:41.652 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/aggregation/elmo_output_ELMo_gamma:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with aggregation/elmo_output_ELMo_gamma
2018-11-16 10:57:41.658 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm/CNN/

2018-11-16 10:57:41.798 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel
2018-11-16 10:57:41.804 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel
2018-11-16 10:57:41.810 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_

2018-11-16 10:57:41.945 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm_1/Variable_5:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm_1/Variable_5
2018-11-16 10:57:41.950 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm_1/Variable_6:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm_1/Variable_6
2018-11-16 10:57:41.957 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm_1/Variable_7:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm_1/Variable_7
2018-11-16 10:57:43.664 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variabl

{"valid": {"eval_examples_count": 403, "metrics": {"roc_auc": 0.6418, "sets_accuracy": 0.6005, "f1_macro": 0.5991}, "time_spent": "0:00:09", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 5}}
{"train": {"epochs_done": 1, "batches_seen": 4, "train_examples_seen": 3613, "metrics": {"roc_auc": 0.5636, "sets_accuracy": 0.5278, "f1_macro": 0.5278}, "time_spent": "0:01:51", "loss": 2.0503659397363663}}


2018-11-16 10:59:44.96 INFO in 'deeppavlov.core.commands.train'['train'] at line 518: New best roc_auc of 0.6867
2018-11-16 10:59:44.99 INFO in 'deeppavlov.core.commands.train'['train'] at line 520: Saving model
2018-11-16 10:59:44.99 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 375: [saving model to /cephfs/home/dilyara.baymurzina/GitHub/reserve/DeepPavlov/download/YahooAnswers/models/model_v39/model_opt.json]


...

Iteration 6: add 1128 samples to train dataset


2018-11-16 16:10:13.97 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /cephfs/home/dilyara.baymurzina/GitHub/reserve/DeepPavlov/download/YahooAnswers/models/model_v39/yahoo_answers_classes.dict]
2018-11-16 16:10:13.121 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 89: [saving vocabulary to /cephfs/home/dilyara.baymurzina/GitHub/reserve/DeepPavlov/download/YahooAnswers/models/model_v39/yahoo_answers_classes.dict]
2018-11-16 16:10:15.709 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/aggregation/elmo_output_ELMo_W:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with aggregation/elmo_output_ELMo_W
2018-11-16 16:10:15.715 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/aggregation/elmo_output_ELMo_gamma:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sbe

2018-11-16 16:10:15.862 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/bias:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/bias
2018-11-16 16:10:15.868 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel
2018-11-16 16:10:15.874 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yaho

2018-11-16 16:10:16.13 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm_1/Variable_4:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm_1/Variable_4
2018-11-16 16:10:16.19 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm_1/Variable_5:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm_1/Variable_5
2018-11-16 16:10:16.25 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm_1/Variable_6:0 from checkpoint b'/home/kuznetsov/models_env/yahoo-sber-questions_elmo/models/mod_from_git/hubs/yahooo-sber-questions_epoches_n_9/variables/variables' with bilm_1/Variable_6
2018-11-16 16:10:16.30 DEBUG in 'tensorflow'['tf_logging'] at line 100: Initialize variable module/bilm_1/Variable_7:0 fr

{"valid": {"eval_examples_count": 403, "metrics": {"roc_auc": 0.9341, "sets_accuracy": 0.8586, "f1_macro": 0.8572}, "time_spent": "0:00:07", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 5}}
